In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader
import spacy
import os
import random
from collections import Counter
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [7]:
# !pip install torch


# ============================================================================
# 1. CHUẨN BỊ DỮ LIỆU
# ============================================================================

In [8]:


class Vocabulary:
    """Xây dựng từ điển cho một ngôn ngữ"""
    def __init__(self, max_vocab_size=10000):
        self.max_vocab_size = max_vocab_size
        self.word2idx = {}
        self.idx2word = {}
        self.word_freq = Counter()

        # Token đặc biệt
        self.PAD_TOKEN = '<pad>'
        self.UNK_TOKEN = '<unk>'
        self.SOS_TOKEN = '<sos>'
        self.EOS_TOKEN = '<eos>'

        self.pad_idx = 0
        self.unk_idx = 1
        self.sos_idx = 2
        self.eos_idx = 3

    def build_vocab(self, sentences):
        """Xây dựng từ điển từ danh sách câu"""
        # Đếm tần suất từ
        for sentence in sentences:
            self.word_freq.update(sentence)

        # Lấy top từ phổ biến
        most_common = self.word_freq.most_common(self.max_vocab_size - 4)

        # Thêm token đặc biệt
        self.word2idx = {
            self.PAD_TOKEN: self.pad_idx,
            self.UNK_TOKEN: self.unk_idx,
            self.SOS_TOKEN: self.sos_idx,
            self.EOS_TOKEN: self.eos_idx
        }

        # Thêm các từ phổ biến
        for idx, (word, freq) in enumerate(most_common, start=4):
            self.word2idx[word] = idx

        # Tạo ánh xạ ngược
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

    def encode(self, sentence):
        """Chuyển câu thành list index"""
        return [self.word2idx.get(word, self.unk_idx) for word in sentence]

    def decode(self, indices):
        """Chuyển list index thành câu"""
        return [self.idx2word.get(idx, self.UNK_TOKEN) for idx in indices]

    def __len__(self):
        return len(self.word2idx)


class TranslationDataset(Dataset):
    """Dataset cho dịch máy"""
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, idx):
        src = self.src_sentences[idx]
        tgt = self.tgt_sentences[idx]

        # Encode thành indices
        src_indices = self.src_vocab.encode(src) + [self.src_vocab.eos_idx]
        tgt_indices = [self.tgt_vocab.sos_idx] + self.tgt_vocab.encode(tgt) + [self.tgt_vocab.eos_idx]

        return torch.LongTensor(src_indices), torch.LongTensor(tgt_indices)


def collate_fn(batch):
    """Collate function để xử lý padding và sorting"""
    src_batch, tgt_batch = zip(*batch)

    # Sắp xếp theo độ dài giảm dần (bắt buộc cho pack_padded_sequence)
    src_lengths = torch.LongTensor([len(s) for s in src_batch])
    sorted_indices = src_lengths.argsort(descending=True)

    src_batch = [src_batch[i] for i in sorted_indices]
    tgt_batch = [tgt_batch[i] for i in sorted_indices]
    src_lengths = src_lengths[sorted_indices]

    # Padding
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return src_padded, src_lengths, tgt_padded

# ============================================================================
# 2. MÔ HÌNH ENCODER-DECODER
# ============================================================================

In [9]:


class Encoder(nn.Module):
    """LSTM Encoder"""
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        """
        Args:
            src: (batch_size, src_len)
            src_lengths: (batch_size,)
        Returns:
            hidden: (num_layers, batch_size, hidden_size)
            cell: (num_layers, batch_size, hidden_size)
        """
        embedded = self.dropout(self.embedding(src))  # (batch, src_len, emb_dim)

        # Pack sequence
        packed = pack_padded_sequence(embedded, src_lengths.cpu(), batch_first=True, enforce_sorted=True)

        # LSTM
        packed_output, (hidden, cell) = self.lstm(packed)

        # hidden: (num_layers, batch, hidden_size)
        # cell: (num_layers, batch, hidden_size)
        return hidden, cell


class Decoder(nn.Module):
    """LSTM Decoder"""
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, cell):
        """
        Args:
            input_token: (batch_size, 1)
            hidden: (num_layers, batch_size, hidden_size)
            cell: (num_layers, batch_size, hidden_size)
        Returns:
            prediction: (batch_size, vocab_size)
            hidden: (num_layers, batch_size, hidden_size)
            cell: (num_layers, batch_size, hidden_size)
        """
        embedded = self.dropout(self.embedding(input_token))  # (batch, 1, emb_dim)

        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output: (batch, 1, hidden_size)

        prediction = self.fc_out(output.squeeze(1))  # (batch, vocab_size)

        return prediction, hidden, cell


# Thay thế class Seq2Seq.forward bằng đoạn này
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.vocab_size

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # Encoder
        hidden, cell = self.encoder(src, src_lengths)

        # Nếu dùng full teacher forcing -> vectorized decoding (1 pass LSTM)
        if teacher_forcing_ratio >= 1.0 - 1e-9:
            # Input to decoder LSTM are the target tokens excluding the last token
            # (we use targets as inputs when teacher forcing)
            # tgt_in: shape (batch, tgt_len-1)
            tgt_in = tgt[:, :-1]  # exclude final <eos> if present
            embedded = self.decoder.embedding(tgt_in)  # (batch, seq_len, emb_dim)
            embedded = self.decoder.dropout(embedded)

            # Run decoder LSTM once for the whole sequence
            decoder_outputs, (hidden, cell) = self.decoder.lstm(embedded, (hidden, cell))
            # decoder_outputs: (batch, seq_len, hidden_size)
            # Map to vocab
            pred = self.decoder.fc_out(decoder_outputs)  # (batch, seq_len, vocab_size)

            # Place predictions into outputs aligned so that outputs[:, t, :] predicts token at tgt[:, t]
            outputs[:, 1:tgt_len, :] = pred  # note: pred corresponds to steps 1..tgt_len-1
            # (outputs[:,0,:] stays zeros or you can set it to prediction for <sos> if you want)
            return outputs

        # Fallback: mixed or no teacher forcing -> step-by-step (cũ)
        decoder_input = tgt[:, 0].unsqueeze(1)  # <sos>
        for t in range(1, tgt_len):
            prediction, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[:, t, :] = prediction
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = prediction.argmax(1).unsqueeze(1)
            decoder_input = tgt[:, t].unsqueeze(1) if teacher_force else top1

        return outputs


# ============================================================================
# 3. HUẤN LUYỆN
# ============================================================================


In [10]:
def train_epoch(model, dataloader, optimizer, criterion, clip, device):
    """Huấn luyện 1 epoch"""
    model.train()
    epoch_loss = 0

    for src, src_lengths, tgt in dataloader:
        src = src.to(device)
        tgt = tgt.to(device)

        optimizer.zero_grad()

        # Forward
        output = model(src, src_lengths, tgt, teacher_forcing_ratio=0.5)

        # output: (batch, tgt_len, vocab_size)
        # tgt: (batch, tgt_len)

        # Bỏ <sos> token và flatten
        output = output[:, 1:, :].reshape(-1, output.shape[-1])
        tgt = tgt[:, 1:].reshape(-1)

        # Tính loss
        loss = criterion(output, tgt)

        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


def evaluate(model, dataloader, criterion, device):
    """Đánh giá trên tập validation"""
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, src_lengths, tgt in dataloader:
            src = src.to(device)
            tgt = tgt.to(device)

            # Forward (không teacher forcing)
            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)

            output = output[:, 1:, :].reshape(-1, output.shape[-1])
            tgt = tgt[:, 1:].reshape(-1)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# ============================================================================
# 4. DỰ ĐOÁN (INFERENCE)
# ============================================================================


In [11]:
def translate_sentence(model, sentence, src_vocab, tgt_vocab, src_tokenizer, device, max_len=50):
    """
    Dịch một câu từ tiếng Anh sang tiếng Pháp

    Args:
        model: mô hình đã train
        sentence: câu tiếng Anh (string)
        src_vocab, tgt_vocab: từ điển
        src_tokenizer: tokenizer cho tiếng Anh
        device: cuda/cpu
        max_len: độ dài tối đa câu dịch

    Returns:
        translated_sentence: câu tiếng Pháp (string)
    """
    model.eval()

    # Tokenize
    tokens = src_tokenizer(sentence.lower())
    tokens = [token.text for token in tokens]

    # Encode
    indices = src_vocab.encode(tokens) + [src_vocab.eos_idx]
    src_tensor = torch.LongTensor(indices).unsqueeze(0).to(device)  # (1, src_len)
    src_lengths = torch.LongTensor([len(indices)])

    with torch.no_grad():
        # Encoder
        hidden, cell = model.encoder(src_tensor, src_lengths)

        # Decoder
        decoder_input = torch.LongTensor([tgt_vocab.sos_idx]).unsqueeze(0).to(device)
        translated_indices = []

        for _ in range(max_len):
            prediction, hidden, cell = model.decoder(decoder_input, hidden, cell)
            predicted_token = prediction.argmax(1).item()

            if predicted_token == tgt_vocab.eos_idx:
                break

            translated_indices.append(predicted_token)
            decoder_input = torch.LongTensor([predicted_token]).unsqueeze(0).to(device)

    # Decode
    translated_tokens = tgt_vocab.decode(translated_indices)
    translated_sentence = ' '.join(translated_tokens)

    return translated_sentence


def calculate_bleu(model, test_data, src_vocab, tgt_vocab, src_tokenizer, device):
    """Tính BLEU score trên tập test"""
    bleu_scores = []
    smoothing = SmoothingFunction().method1

    for src_sent, tgt_sent in test_data:
        src_text = ' '.join(src_sent)
        translated = translate_sentence(model, src_text, src_vocab, tgt_vocab, src_tokenizer, device)

        reference = [tgt_sent]
        candidate = translated.split()

        score = sentence_bleu(reference, candidate, smoothing_function=smoothing)
        bleu_scores.append(score)

    return np.mean(bleu_scores)


# ============================================================================
# 5. MAIN - SỬ DỤNG
# ============================================================================

In [12]:
def main():
    """Hàm chính - ví dụ sử dụng. Ở đây thêm code load/tokenize/build vocab và tạo DataLoader."""
    # Cấu hình
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hyperparameters
    EMBEDDING_DIM = 256
    HIDDEN_SIZE = 512
    NUM_LAYERS = 2
    DROPOUT = 0.5
    LEARNING_RATE = 0.001
    BATCH_SIZE = 64
    NUM_EPOCHS = 10
    CLIP = 1

    # -- Đường dẫn tới thư mục dữ liệu (tương đối với notebook) --
    # data_dir = os.path.join(os.getcwd(), 'dataset') # Original incorrect path
    data_dir = '/content/' # Correct path to the dataset files

    def load_lines(path):
        """Đọc file và trả về danh sách dòng (không tokenized)"""
        lines = []
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                if line:
                    lines.append(line)
        return lines

    def tokenize_lines(lines, tokenizer):
        """Tokenize danh sách câu trả về list of token lists"""
        tokenized = []
        for line in lines:
            toks = [t.text for t in tokenizer(line.lower()) if t.text.strip()]
            tokenized.append(toks)
        return tokenized

    # Tập tin theo cấu trúc workspace: dataset/<split>/(file)
    # Updated paths to reflect the file location in /content/
    train_en_path = os.path.join(data_dir, 'train.en')
    train_fr_path = os.path.join(data_dir, 'train.fr')
    val_en_path = os.path.join(data_dir, 'val.en')
    val_fr_path = os.path.join(data_dir, 'val.fr')
    test_en_path = os.path.join(data_dir, 'test_2018_flickr.en')
    test_fr_path = os.path.join(data_dir, 'test_2018_flickr.fr')


    # Load raw lines
    train_en_lines = load_lines(train_en_path)
    train_fr_lines = load_lines(train_fr_path)
    val_en_lines = load_lines(val_en_path)
    val_fr_lines = load_lines(val_fr_path)
    test_en_lines = load_lines(test_en_path)
    test_fr_lines = load_lines(test_fr_path)

    # Tokenizers: thử load spacy models, fallback sang spacy.blank nếu model chưa cài
    try:
        en_tokenizer = spacy.load('en_core_web_sm')
    except Exception:
        # Nếu model en_core_web_sm chưa cài, dùng blank tokenizer (đơn giản)
        print("Warning: en_core_web_sm not found. Using spacy.blank('en'). Please install with !python -m spacy download en_core_web_sm")
        en_tokenizer = spacy.blank('en')

    try:
        fr_tokenizer = spacy.load('fr_core_news_sm')
    except Exception:
        print("Warning: fr_core_news_sm not found. Using spacy.blank('fr'). Please install with !python -m spacy download fr_core_news_sm")
        fr_tokenizer = spacy.blank('fr')

    # Tokenize tất cả
    print('Tokenizing...')
    train_en_tok = tokenize_lines(train_en_lines, en_tokenizer)
    train_fr_tok = tokenize_lines(train_fr_lines, fr_tokenizer)
    val_en_tok = tokenize_lines(val_en_lines, en_tokenizer)
    val_fr_tok = tokenize_lines(val_fr_lines, fr_tokenizer)
    test_en_tok = tokenize_lines(test_en_lines, en_tokenizer)
    test_fr_tok = tokenize_lines(test_fr_lines, fr_tokenizer)


    # Build vocabularies (từ train set)
    print('Building vocabularies...')
    src_vocab = Vocabulary(max_vocab_size=10000)
    src_vocab.build_vocab(train_en_tok)
    tgt_vocab = Vocabulary(max_vocab_size=10000)
    tgt_vocab.build_vocab(train_fr_tok)

    # Tạo Dataset và DataLoader
    train_dataset = TranslationDataset(train_en_tok, train_fr_tok, src_vocab, tgt_vocab)
    val_dataset = TranslationDataset(val_en_tok, val_fr_tok, src_vocab, tgt_vocab)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # test_data dùng cho calculate_bleu (dạng list of (src_tokens, tgt_tokens))
    test_data = list(zip(test_en_tok, test_fr_tok))

    # Sanity prints
    print(f'Train examples: {len(train_dataset)}')
    print(f'Val examples: {len(val_dataset)}')
    print(f'Test examples: {len(test_data)}')
    print('Example tokenized (src -> tgt):')
    print(train_en_tok[0][:20])
    print(train_fr_tok[0][:20])

    # Khởi tạo mô hình (dùng kích thước từ vocab đã tạo)
    encoder = Encoder(len(src_vocab), EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    decoder = Decoder(len(tgt_vocab), EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    model = Seq2Seq(encoder, decoder, device).to(device)

    # Optimizer và loss
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # Use ignore_index for PAD_TOKEN in CrossEntropyLoss
    criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab.pad_idx) # Corrected to use tgt_vocab.pad_idx


    # (Phần training vẫn như cũ) -- user có thể chạy training nếu muốn
    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0

    for epoch in range(NUM_EPOCHS):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, CLIP, device)
        val_loss = evaluate(model, val_loader, criterion, device)

        print(f'Epoch {epoch+1}/{NUM_EPOCHS}')
        print(f'Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping!")
                break

    # Đánh giá BLEU
    bleu_score = calculate_bleu(model, test_data, src_vocab, tgt_vocab, en_tokenizer, device)
    print(f'BLEU Score: {bleu_score:.4f}')

    # Ví dụ dịch
    sentence = "A man is riding a horse."
    translation = translate_sentence(model, sentence, src_vocab, tgt_vocab, en_tokenizer, device)
    print(f'English: {sentence}')
    print(f'French: {translation}')

In [13]:
if __name__ == '__main__':
    main()

Tokenizing...
Building vocabularies...
Train examples: 29000
Val examples: 1014
Test examples: 1071
Example tokenized (src -> tgt):
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']
['deux', 'jeunes', 'hommes', 'blancs', 'sont', 'dehors', 'près', 'de', 'buissons', '.']
Epoch 1/10
Train Loss: 4.933 | Val Loss: 4.712
Epoch 2/10
Train Loss: 3.970 | Val Loss: 4.344
Epoch 3/10
Train Loss: 3.548 | Val Loss: 4.071
Epoch 4/10
Train Loss: 3.269 | Val Loss: 3.937
Epoch 5/10
Train Loss: 3.023 | Val Loss: 3.897
Epoch 6/10
Train Loss: 2.841 | Val Loss: 3.745
Epoch 7/10
Train Loss: 2.678 | Val Loss: 3.698
Epoch 8/10
Train Loss: 2.560 | Val Loss: 3.611
Epoch 9/10
Train Loss: 2.428 | Val Loss: 3.640
Epoch 10/10
Train Loss: 2.328 | Val Loss: 3.601
BLEU Score: 0.1279
English: A man is riding a horse.
French: un homme surfe un cheval .


In [14]:
import torch
print(torch.cuda.is_available())


True


In [15]:
!nvidia-smi


Tue Nov  4 17:16:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             36W /   70W |    4850MiB /  15360MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
# !python -m spacy download fr_core_news_sm